In [1]:
import PyPDF2
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import os
import re
import itertools
#source: https://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import logging
from datetime import datetime
import csv

# Text extraction

In [2]:
#read and extract text from one pdf file using pypdf2
def convertPdfToTxt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
def readPDF(fullPath):
    pdf_file = open(fullPath, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    page = read_pdf.getPage(0)
    page_content = page.extractText()
    return page_content

In [9]:
file = 'D:/Users/figohjs/Documents/JOPPK/Data/Raw/Quarterly Earnings PDF 4Q 2019/Materials/600585 CG 9M2019 Earnings Update.pdf'
a = convertPdfToTxt(file) 

# Extracting metadata

In [12]:
period = re.search('(.{2} )\d{4}.*', file).group(1)
period
year = re.search('(\d{4}) (Earnings|Sales|Revenue).*', file).group(1)
year

'2019'

# Group Text

In [66]:
#group content - pdfminer based on results review, assessment, more details 
def groupText(content):
    #topicList = ['RESULTS REVIEW:', 'ASSESSMENT:', 'MORE DETAILS:']
    topicList = ['RAHSIA', 'ASSESSMENT:', 'MORE DETAILS:']
    contentList = content.split('\n')
    if topicList[0] in content:
        review = ' '.join([i.strip() for i in contentList[contentList.index(topicList[0]) + 1:contentList.index(topicList[1])]
                          if i not in  ['', 'RESULTS REVIEW:']])
    elif 'RESULTS REVIEW:' in content:
        review = ' '.join([i.strip() for i in contentList[contentList.index('RESULTS REVIEW:') + 1:contentList.index(topicList[1])]
                          if i not in  ['', 'RESULTS REVIEW:']])        
    assessment = ' '.join([i.strip() for i in contentList[contentList.index(topicList[1]) + 1:contentList.index(topicList[2]) - 1]
                      if i not in ['', ':', 'comments:']])
    details = ' '.join([i.strip() for i in contentList[contentList.index(topicList[2]):-2]
                       if i not in ['',':']])
    if 'Announcement date:' in contentList[-1]:
        announcementDate = re.search('Announcement date: (.+?\d{4})', contentList[-1]).group(1)
    else:
        announcementDate = ''
        
    return review, assessment, details, announcementDate

In [34]:
def groupText2(content):
    topicList = ['RESULTS REVIEW:', 'ASSESSMENT:', 'MORE DETAILS:']

    contentList = content.split('\n')
    review = ' '.join([i.strip() for i in contentList[contentList.index(topicList[0]) + 1:contentList.index(topicList[1])]
                      if i!= ''])
    assessment = ' '.join([i.strip() for i in contentList[contentList.index(topicList[1]) + 1:contentList.index(topicList[2]) - 1]
                      if i not in ['', ':', 'comments:']])
    contentList2 = [i for i in contentList if i != '']
    details = ' '.join([re.sub('Ł', '', i.strip()) for i in contentList2[contentList2.index(topicList[2]) + 
                                      1:] if i not in [':']])
    if 'Announcement date:' in contentList[-1]:
        announcementDate = re.search('Announcement date: (.+?\d{4})', groupText(pdf2)[-1]).group(1)
    else:
        announcementDate = ''
    return review, assessment, details, announcementDate

In [68]:
path = '../Data/Raw/Quarterly Earnings PDF 2018/Consumer Discretionary/2Q/DAI GY Q2 2018 Earnings Update.pdf'

pdf = convertPdfToTxt(path)

In [78]:
path = '../Data/Test/XOM UN Q1 2019 Earnings Update.pdf' 

pdf2 = convertPdfToTxt(path)

#pdf2 = readPDF(path)

In [79]:
pdf2

'RAHSIA\n\nRESULTS REVIEW:\n\n• Miss: Q1 2019 earnings miss analysts’ estimates with adjusted EPS of USD0.55 (-23% from consensus USD0.72) driven\nby the downstream segment as narrowed refinery margins and higher maintenance pressured earnings.\n\n• Share price reaction: Shares of ExxonMobil fell 2.1% following its weak earnings and lower oil prices on the day (Brent\ncrude -3.0%, S&P Energy index -1.2%). \n\nASSESSMENT:\n\n• Among integrated oil companies, XOM has a clear strategy with assets in three key areas; US shale (Permian basin),\nLNG (Papua New Guinea and Mozambique) and deepwater (Guyana, Brazil). Unlike peers however, XOM is pursuing an\naggressive growth strategy, increasing capex targetting production of ~5mb/d by 2025 (current production 4.0mb/d). If\nsuccessful, cash flow generation from these long-dated projects will be positive for the stock, however lower oil prices\nremain a significant risk.\n\n• Analysts’ comments: JPM noted that Exxon continues to make progress o

In [84]:
pd.to_datetime(groupText(pdf2)[-1]).year

2019

In [70]:
groupText(pdf)

('• Missed: Daimler reported EBIT 8% below consensus. EBIT margin of 7.5% was below consensus of 7.8%, while 2Q2018 EPS of EUR1.61 was also slightly lower than consensus of EUR1.63 due to weaker car division, including China import tariff issue, lost production in the US and weaker diesel pricing. Daimler cut its units sales forecast for Mercedes cars and warned that 3Q profitability to be significantly below 2Q mainly due to new emission testing regulation in the Europe (WLTP). • Share price reaction: Shares fell 0.34% after the results were released.',
 '• Going forward, developments in its Truck division spin-offs would be key stock price catalysts as its new model launch cycle has peaked. • Analysts’ comments: With the ongoing headwinds (WLTP and tariffs) and the best of the MB Cars cycle having passed, the outlook for EPS will remain subdued at best. Table: Q2 2018 Results',
 'MORE DETAILS: • In light of a longer certification process, Daimler has reduced the unit sales outlook fo

In [44]:
if 'Announcement date:' in groupText(pdf2)[-1]:
    print(pd.to_datetime(re.search('Announcement date: ([^a-zA-Z]+)', groupText(pdf2)[-1]).group(1)).month)

RecursionError: maximum recursion depth exceeded in comparison

In [27]:
if pd.isnull(pd.to_datetime('')):
    print('Yes')

Yes


In [13]:
groupText(pdf2)[-1]

'(USD mn)Q1 2019Q4 2018FY2018FY2019eFY2020eFY2021eRevenue58,64068,253279,332291,518305,036279,872Growth (YoY)-10.4%4.5%17.8%4.4%4.6%-8.2%EBITNA4,69420,83930,92737,67933,806Growth (YoY)NA134.7%72.6%48.4%21.8%-10.3%MarginNA6.9%7.5%10.6%12.4%12.1%Net income2,3506,00020,84018,30723,52222,020Growth (YoY)-49.5%-28.4%5.7%-12.2%28.5%-6.4%Margin4.0%8.8%7.5%6.3%7.7%7.9%Source: BloombergEXXON MOBIL CORPQ1 2019 Earnings Update (FY-end 31 December)Announcement date: 26/04/2019Holdings InformationCurrent Price (28-Apr-2019)USD80.49Original Book Cost (USDm)135.620192020202112MUpsideMTM vs Ori (% / USDm)-9.0% / -12.1Internal Price Target77.8781.8781.39External Price Target83.674%MTM vs Imp (% / USDm)8.3% / 9.4Upside-3%2%1%Citi75.00-7%YTD Price Performance18%PE17.5017.0017.00MACQ89.0011%Dividend Yield (Cost)4.3%EPS4.454.824.79JPM87.008%\x0c'